In [5]:
import whisper
import datasets
import threading
import pydub
import soundfile
import pandas as pd
import numpy as np
import librosa
import os
from pydub import AudioSegment
import os
import whisper
from pydub import AudioSegment
import datasets
import pandas as pd
import concurrent.futures

In [6]:
dataset = datasets.load_dataset("CAiRE/ASCEND")

In [7]:
# Function to separate mp3 file by timestamp
set = pd.DataFrame(dataset['train'])
display(set)

,id,path,audio,transcription,duration,language,original_speaker_id,session_id,topic
0,00000,C:\Users\Owner\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\Owner\.cache\huggingface\da...,我刚刚开始record,1.560,mixed,1,1,persona
1,00001,C:\Users\Owner\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\Owner\.cache\huggingface\da...,嗯hello我的名字叫徐妍,4.160,mixed,1,1,persona
2,00002,C:\Users\Owner\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\Owner\.cache\huggingface\da...,嗯初次见面nice to meet you嗯,3.320,mixed,1,1,persona
3,00003,C:\Users\Owner\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\Owner\.cache\huggingface\da...,今天呢我非常希望能够通过这个机会去跟你make friends,5.700,mixed,1,1,persona
4,00004,C:\Users\Owner\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\Owner\.cache\huggingface\da...,嗯你知道就是,2.020,zh,1,1,persona
...,...,...,...,...,...,...,...,...,...
9864,09864,C:\Users\Owner\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\Owner\.cache\huggingface\da...,嗯,0.550,zh,8,4,technology
9865,09865,C:\Users\Owner\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\Owner\.cache\huggingface\da...,her intellect intelli,1.485,en,8,3,education
9866,09866,C:\Users\Owner\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\Owner\.cache\huggingface\da...,学术方面的话 其实国内也有很多 学术很厉害 的 嗯 professor之类的 但是可能整体的...,14.506,mixed,8,3,education
9867,09867,C:\Users\Owner\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\Owner\.cache\huggingface\da...,home school education can provide an environme...,8.611,en,8,3,education


In [8]:
path = set.at[9595, 'path']
print(path)

C:\Users\Owner\.cache\huggingface\datasets\downloads\extracted\bb19ffdd43a09a30bc7580ba3c1fbbdaae5c86c6fca95346cc7ea64d036e35d1\waves/ses4_spk26_L24529_475.700_8.140.wav


In [12]:
for i in range(100):
    path = set.at[i, 'path']


In [26]:
model = whisper.load_model("base")

def split_audio(file, start, end):
    wav = AudioSegment.from_wav(file)
    first_half = wav[start:end]
    second_half = wav[end:]
    first_half.export("first_half_"+file, format="wav")
    second_half.export("second_half_"+file, format="wav")

def get_highest_hi_en(file):
    audio = whisper.load_audio(file)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
    sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]
    hi_prob = probs['hi']
    en_prob = probs['en']
    if hi_prob > en_prob:
        return {'key': 'hi','value': probs['hi']}
    else:
        return {'key': 'en', 'value': probs['en']}

def get_highest_whisper_entry(file):
    audio = whisper.load_audio(file)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
    sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]
    return sorted_probs[0]

def binary_search(file, start, end, new_file, my_dict):
    if get_highest_hi_en(new_file)['value'] > 0.6:
        my_dict[start] = get_highest_hi_en(new_file)['key']
        print('high prob of', get_highest_hi_en(new_file)['value'] ,'that its 1 language:', my_dict)
        return my_dict
    
    if end - start <= 2000:
        my_dict[start] = get_highest_hi_en(new_file)['key']
        print('clip too short, split with prob:', get_highest_hi_en(new_file)['value'], my_dict)
        return
    
    split_position = start + np.ceil((end - start) / 2)
    split_position = int(split_position)
    split_audio(file, start, split_position)
    binary_search(file, start, split_position, "first_half_"+file, my_dict)
    
    # Update split position for the second half
    split_position = int(split_position)
    split_audio(file, split_position, end)
    binary_search(file, split_position, end, "second_half_"+file, my_dict)

    return my_dict

results = []

def chunk_audio_30_sec(file):
    files = []
    wav_file = AudioSegment.from_wav(file)
    duration = len(wav_file)
    for i in range(0, duration, 30000):
        chunk = wav_file[i:i+29999]
        name = f"{int(i/30000)}.wav"
        chunk.export(name, format="wav")
        files.append(name)
    return files

def get_whisper_translation(translated_texts, audio, curr_language):
    model = whisper.load_model("base")
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    options = whisper.DecodingOptions(language=curr_language, task='translate', fp16=False)
    result = whisper.decode(model, mel, options)
    translated_texts.append(result.text)    

def translate_audio(audio_path, segments_dict, model_name="base"):
    model = whisper.load_model(model_name)
    full_audio = AudioSegment.from_file(audio_path, format="wav")

    translated_texts = []
    start_time = 0  # Start of the first segment
    current_language = list(segments_dict.values())[0]  # Language of the first segment

    for end_time, language in segments_dict.items():
        if language != current_language:
            # Process the accumulated segment in the current language
            start_ms = start_time
            end_ms = end_time - 1  # Adjusting to include the last moment of the previous language

            audio_segment = full_audio[start_ms:end_ms]
            temp_file = "temp_segment.wav"
            audio_segment.export(temp_file, format="wav")

            audio = whisper.load_audio(temp_file)
            get_whisper_translation(translated_texts, audio, current_language)

            os.remove(temp_file)

            # Update the start time and current language
            start_time = end_time
            current_language = language

    # Process the final segment
    start_ms = start_time
    end_ms = len(full_audio)  # Till the end of the adio
    audio_segment = full_audio[start_ms:end_ms]
    temp_file = "temp_segment.wav"
    audio_segment.export(temp_file, format="wav")
    audio = whisper.load_audio(temp_file)
    get_whisper_translation(translated_texts, audio, current_language)
    os.remove(temp_file)

    full_translation = ' '.join(translated_texts)
    return full_translation

my_dict = {}

In [27]:
chunk_audio_30_sec('combined_audio.wav')
binary_search("1.wav", 0, len(AudioSegment.from_wav("1.wav")), "1.wav", my_dict)
translate_audio("1.wav", my_dict)

KeyboardInterrupt: 

In [28]:
binary_search("alia_30s_clip_2.wav", 0, len(AudioSegment.from_wav("alia_30s_clip_2.wav")), "alia_30s_clip_2.wav", my_dict)
translate_audio("alia_30s_clip_2.wav", my_dict)

clip too short, split with prob: 0.4205993711948395 {0: 'en'}
high prob of 0.7981908321380615 that its 1 language: {0: 'en', 1875: 'hi'}
high prob of 0.8285928964614868 that its 1 language: {0: 'en', 1875: 'hi', 3750: 'hi'}
high prob of 0.7813481688499451 that its 1 language: {0: 'en', 1875: 'hi', 3750: 'hi', 7500: 'hi'}
high prob of 0.9189804792404175 that its 1 language: {0: 'en', 1875: 'hi', 3750: 'hi', 7500: 'hi', 15000: 'hi'}
high prob of 0.8299621343612671 that its 1 language: {0: 'en', 1875: 'hi', 3750: 'hi', 7500: 'hi', 15000: 'hi', 22500: 'hi'}
high prob of 0.9538677930831909 that its 1 language: {0: 'en', 1875: 'hi', 3750: 'hi', 7500: 'hi', 15000: 'hi', 22500: 'hi', 26250: 'hi'}
clip too short, split with prob: 0.3625158965587616 {0: 'en', 1875: 'hi', 3750: 'hi', 7500: 'hi', 15000: 'hi', 22500: 'hi', 26250: 'hi', 28125: 'en'}


"of the investment that I have been making. If you want to make something 4 years from a 2 year old age, it means that you want to make a major difference. So when it's actually living your life, I think there are no complaints. Which is what is happening with me right now. I always wanted to make an actress, I made a film, I didn't know what was going on at that time. I just realized that..."